In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv("D:/추천알고리즘강의/data/ml-latest-small/ratings.csv")

In [3]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [4]:
from datetime import datetime

In [5]:
ratings['timestamp']=ratings['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

In [6]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-31 03:45:03
1,1,3,4.0,2000-07-31 03:20:47
2,1,6,4.0,2000-07-31 03:37:04
3,1,47,5.0,2000-07-31 04:03:35
4,1,50,5.0,2000-07-31 03:48:51
...,...,...,...,...
100831,610,166534,4.0,2017-05-04 06:53:22
100832,610,168248,5.0,2017-05-04 07:21:31
100833,610,168250,5.0,2017-05-09 04:50:47
100834,610,168252,5.0,2017-05-04 06:19:12


In [7]:
ratings.to_csv("D:/추천알고리즘강의/data/ml-latest-small/ratings_update.csv")

In [8]:
ratings = pd.read_csv("D:/추천알고리즘강의/data/ml-latest-small/ratings_update.csv")

In [9]:
ratings

,Unnamed: 0,userId,movieId,rating,timestamp
0,0,1,1,4.0,2000-07-31 03:45:03
1,1,1,3,4.0,2000-07-31 03:20:47
2,2,1,6,4.0,2000-07-31 03:37:04
3,3,1,47,5.0,2000-07-31 04:03:35
4,4,1,50,5.0,2000-07-31 03:48:51
...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-04 06:53:22
100832,100832,610,168248,5.0,2017-05-04 07:21:31
100833,100833,610,168250,5.0,2017-05-09 04:50:47
100834,100834,610,168252,5.0,2017-05-04 06:19:12


## RMSE

In [10]:
ratings_example = [[4, 3.5], [5, 5], [0.5, 1], [3, 5]]
ratings_example =pd.DataFrame(ratings_example, columns=['Actual', 'Predict'])

In [11]:
ratings_example['error']=ratings_example['Actual']-ratings_example['Predict']

In [12]:
# squared error의 캔슬되는 것을 막기위해 제곱
ratings_example['squared error']=ratings_example['error']**2

In [13]:
mse = ratings_example['squared error'].mean()

In [14]:
import numpy as np

In [15]:
# root mean squared error : rmse
rmse = np.sqrt(mse)
rmse

1.0606601717798212

## RMSE with sklearn

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
mse = mean_squared_error(ratings_example['Actual'], ratings_example['Predict'])
rmse = np.sqrt(mse)
rmse

1.0606601717798212

## Train Test Split

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train, test = train_test_split(ratings, random_state=27, test_size=0.1)

In [20]:
print(train.shape)
print(test.shape)

(90752, 5)
(10084, 5)


## 가장 간단한 예측하기

### 0.5점으로 예측하기

In [21]:
prediction = [0.5] * len(test)

In [22]:
mse1 = mean_squared_error(test['rating'], prediction)

In [23]:
rmse1 = np.sqrt(mse1)
rmse1

3.1604543718542732

### 모두 5점으로 예측하기

In [24]:
prediction = [5.0]*len(test)

In [25]:
mse1 = mean_squared_error(test['rating'], prediction)

In [26]:
rmse1 = np.sqrt(mse1)
rmse1

1.8345717512574158

### 데이터의 평균으로 예측하기
#### train 데이터의 평균으로  test 데이터의 평균 예측하기

In [27]:
prediction = [train['rating'].mean()]*len(test)
mse1 = mean_squared_error(test['rating'], prediction)

In [28]:
rmse1 = np.sqrt(mse1)
rmse1

1.0360327132036247

### 사용자 평점 기반 예측하기 :
#### 각 사용자의 평균 평점으로(train), 영화에 대한 평점을 예측(test)하세요.

* 예시: userId 1번의 train 데이터 평균 평점이 4.33이라면, test 데이터에 있는 userId 1번의 모든 영화에 대한 평점을 4.33이라고 예측한다.

* train에 해당 사용자에 대한 평점 기록이 전혀 없다면, 어떻게 해야 할까요?

* 이 예측 방법은 어떤 가정을 가지고 있나요? 예측의 근거를 찾아 보세요. 

In [33]:
users = train.groupby('userId')['rating'].mean().reset_index()
users = users.rename(columns={'rating': 'predict'})
users[:5]

,userId,predict
0,1,4.383562
1,2,3.928571
2,3,2.787879
3,4,3.520833
4,5,3.641026


In [35]:
predict_by_users=test.merge(users, how='left', on='userId')

In [39]:
predict_by_users.isnull().sum()

Unnamed: 0    0
userId        0
movieId       0
rating        0
timestamp     0
predict       0
dtype: int64

In [41]:
mse=mean_squared_error(predict_by_users['rating'], predict_by_users['predict'])

In [42]:
rmse = np.sqrt(mse)
rmse

0.9433592064402052

## 예측의 근거

In [43]:
train['rating'].std()

1.0432502359195306

In [45]:

train.groupby("userId")['rating'].std().mean()

0.9239208986877873

### 영화 평점 기반 예측하기
* 각 영화의 평균 평점으로(train), 영화에 대한 평점을 예측(test)하세요.

* 예시: movieId 1번의 train 데이터 평균 평점이 4.5라면, test 데이터에 있는 movieId 1번 영화에 대한 모든 사용자의 평점을 4.5라고 예측한다.

* train에 해당 영화에 대한 평점 기록이 전혀 없다면, 어떻게 해야 할까요?

* 이 예측 방법은 어떤 가정을 가지고 있나요? 예측의 근거를 찾아 보세요. 

In [49]:
movie_rating = train.groupby('movieId')['rating'].mean().reset_index()
movie_rating = movie_rating.rename(columns={'rating':'predict'})

In [51]:
movie_rating_use = test.merge(movie_rating, how='left', on='movieId')

In [52]:
movie_rating_use

,Unnamed: 0,userId,movieId,rating,timestamp,predict
0,34844,233,32587,3.0,2018-08-03 20:41:54,3.817568
1,9115,62,135143,4.0,2018-03-20 05:02:34,3.636364
2,75330,475,103042,4.0,2017-06-21 16:47:28,3.266667
3,44282,294,2628,4.0,2000-08-19 06:27:10,3.133858
4,59001,385,288,1.0,1996-06-14 03:47:22,3.283951
...,...,...,...,...,...,...
10079,44675,298,2325,0.5,2016-07-03 04:59:52,3.333333
10080,22134,146,2805,2.5,2011-08-05 10:30:35,2.852941
10081,74127,474,3702,2.0,2004-07-10 00:37:24,3.600000
10082,48781,316,3968,2.5,2005-03-22 21:13:05,3.100000


In [63]:
movie_rating_use.isnull().sum()

Unnamed: 0    0
userId        0
movieId       0
rating        0
timestamp     0
predict       0
dtype: int64

In [62]:
movie_rating_use.loc[movie_rating_use['predict'].isnull(), 'predict'] = train['rating'].mean()

In [64]:
mse = mean_squared_error(movie_rating_use['rating'], movie_rating_use['predict'])
mse

0.9355720568364707

In [65]:
np.sqrt(mse)

0.9672497386076001